![cover](./images/cover.png)

In [1]:
import QuantLib as ql
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
from datetime import datetime

from plotly.offline import init_notebook_mode
# from IPython.display import display, HTML
import plotly

import plotly.offline as po
import plotly.graph_objs as go
init_notebook_mode(connected=True)

In [3]:
def getPillar(label):
    numList = [int(s) for s in re.findall(r'\d+',label)]
    p = label[-1]
    if (p == 'N'):
        return ql.Period(1, ql.Days)
    elif (p == 'W'):
        n = numList[0]
        return ql.Period(n, ql.Weeks)
    elif(p == 'M'):
        n = numList[0]
        return ql.Period(n, ql.Months)
    else:
        n = numList[0]
        return ql.Period(n, ql.Years)
def long_to_date (longDate):
    return datetime.fromordinal(datetime(1900, 1, 1).toordinal() + longDate - 2).strftime('%d/%m/%Y')

In [4]:
today = ql.Date(15,ql.January, 2021)
ql.Settings.instance().evaluationDate = today

In [5]:
EONIA_helpers = []
url_EONIA = './EUROIS.csv'
dataset_EONIA = pd.read_csv(url_EONIA)
for idx, row in dataset_EONIA.iterrows():
    rate = row['Market quote']
    fixingDays = row['FixDays']
    period = getPillar(row['Maturity'])
    if(row['Tp'] == 'Dp'):
        EONIA_helpers +=[
                        ql.DepositRateHelper(
                            ql.QuoteHandle(ql.SimpleQuote(rate)),
                            period, fixingDays,
                            ql.TARGET(), ql.Following,
                            False, ql.Actual360())]
eonia = ql.Eonia()
for idx, row in dataset_EONIA.iterrows():
    rate = row['Market quote']
    fixingDays = row['FixDays']
    period = getPillar(row['Maturity'])
    if(row['Tp'] == 'Sw'):
        EONIA_helpers +=[
                        ql.OISRateHelper(fixingDays, period,
                                         ql.QuoteHandle(ql.SimpleQuote(rate)), eonia)]

In [6]:
Curve1 = ql.PiecewiseCubicZero(0,ql.TARGET(), EONIA_helpers, ql.Actual365Fixed())
Curve1.enableExtrapolation()
curve1 = ql.RelinkableYieldTermStructureHandle(Curve1)
Curve2 = ql.PiecewiseLinearZero(0,ql.TARGET(), EONIA_helpers, ql.Actual365Fixed())
Curve2.enableExtrapolation()
curve2 = ql.RelinkableYieldTermStructureHandle(Curve2)

In [7]:
today = curve1.referenceDate()
end = today + ql.Period(60,ql.Years)
dates = [ ql.Date(serial) for serial in range(today.serialNumber(),
end.serialNumber()+1) ]
dates=[d for d in dates if ql.TARGET().isBusinessDay(d)]
pillars = [p for p in Curve1.dates()]
dat = [long_to_date(date.serialNumber()) for date in dates]
pil = [long_to_date(date.serialNumber()) for date in pillars]
discounts = [curve1.discount(d) for d in dates]
zero_rate = [ curve1.zeroRate(ql.TARGET().advance(d,1,ql.Days), ql.Actual365Fixed(), ql.Simple).rate()
for d in dates ]
fwd_rate = [curve1.forwardRate(d, ql.TARGET().advance(d,1,ql.Days),ql.Actual360(), ql.Simple).rate() for d in dates]

discounts2 = [curve2.discount(d) for d in dates]
zero_rate2 = [ curve2.zeroRate(ql.TARGET().advance(d,1,ql.Days), ql.Actual365Fixed(), ql.Simple).rate()
for d in dates ]
fwd_rate2 = [curve2.forwardRate(d, ql.TARGET().advance(d,1,ql.Days),ql.Actual360(), ql.Simple).rate() for d in dates]

pillar_disc = [curve1.discount(d) for d in pillars]
pillar_zero_rate = [ curve1.zeroRate(ql.TARGET().advance(d,1,ql.Days), ql.Actual365Fixed(), ql.Simple).rate()
for d in pillars ]
pillar_fwd_rate = [curve1.forwardRate(d, ql.TARGET().advance(d,1,ql.Days),ql.Actual360(), ql.Simple).rate() for d in pillars]


In [8]:
fig = go.FigureWidget()
#Zero Coupon Rates
fig.add_trace(go.Scatter(x=dat, y=zero_rate,name='Cubic interpolation',visible=True,line={'color':'tomato'}))
fig.add_trace(go.Scatter(x=dat, y=zero_rate2,name='Linear interpolation',visible=True,line={'color':'turquoise'}))
fig.add_trace(go.Scatter(x=pil, y=pillar_zero_rate,name='Pillars',mode='markers',visible=True,marker={'color':'steelblue'}))
#Discount Factors
fig.add_trace(go.Scatter(x=dat, y=discounts,name='Cubic interpolation',visible=False,line={'color':'tomato'}))
fig.add_trace(go.Scatter(x=dat, y=discounts2,name='Linear interpolation',visible=False,line={'color':'steelblue'}))
fig.add_trace(go.Scatter(x=pil, y=pillar_disc,name='Pillars',mode='markers',visible=False,marker={'color':'turquoise'}))
#O/N Libor rates
fig.add_trace(go.Scatter(x=dat, y=fwd_rate,name='Cubic interpolation',visible=False,line={'color':'tomato'}))
fig.add_trace(go.Scatter(x=dat, y=fwd_rate2,name='Linear interpolation',visible=False,line={'color':'steelblue'}))
fig.add_trace(go.Scatter(x=pil, y=pillar_fwd_rate,name='Pillars',mode='markers',visible=False,marker={'color':'turquoise'}))

fig.layout.showlegend = True
fig['layout']['xaxis']={'dtick':2560,'title':'Maturity','titlefont':{'size': 10}}
fig['layout']['yaxis']={'fixedrange': False,'tickformat':',.1p','hoverformat':',.4p','title':'Zero Coupon Rate','titlefont':{'size': 10}}
fig['layout']['height'] = 700
fig['layout']['width'] = 1100
fig['layout']['hovermode'] = 'x'
fig.layout.title={'text':'Interpolation\'s effects on interest rate curves','font':{'size': 25},'x':0.5,'xanchor':'center'}

# Add dropdown
fig.update_layout(
    updatemenus=[
        dict(
            buttons=list([
                dict(
                    args=[{'visible':[True,True,True,False,False,False,False,False,False]},
                         {'yaxis':{'fixedrange': False,
                                   'tickformat':',.1p',
                                   'hoverformat':',.4p',
                                   'title':'Zero Coupon Rate',
                                   'titlefont':{'size': 10}}}],
                    label="Zero Coupon Rates",
                    method="update"
                ),
                dict(
                    args=[{'visible':[False,False,False,True,True,True,False,False,False]},
                         {'yaxis':{'fixedrange': False,
                                   'tickformat':',.2f',
                                   'hoverformat':',.5f',
                                   'title':'Discount Factor',
                                   'titlefont':{'size': 10}}}],
                    label="Discount Factors",
                    method="update"
                ),
                dict(
                    args=[{'visible':[False,False,False,False,False,False,True,True,True]},
                         {'yaxis':{'fixedrange': False,
                                   'tickformat':',.1p',
                                   'hoverformat':',.4p',
                                   'title':'Libor Rate',
                                   'titlefont':{'size': 10}}}],
                    label="Libor Rates",
                    method="update"
                )
            ]),
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.1,
            xanchor="left",
            y=1.1,
            yanchor="top"
        ),
    ]
)

FigureWidget({
    'data': [{'line': {'color': 'tomato'},
              'name': 'Cubic interpolation',
       …

In [9]:
# Add dropdown
fig.update_layout(legend={'orientation':'h'},
    annotations=[dict(text="Curve type", showarrow=False,x=0.06, y=1.075, yref="paper", align="left")])
fig 

FigureWidget({
    'data': [{'line': {'color': 'tomato'},
              'name': 'Cubic interpolation',
       …